# Problem Set 2

In [1]:
# imports
import numpy as np
from importlib import reload

import unyt

from oceanpy import constants
from oceanpy.forces import coriolis

from os_classes.ocea257 import chapter1, chapter2
from os_classes import plotting as os_plotting


from bokeh import plotting
from bokeh import models

# Analytic

## PS2-1 (7-1) -- Geostrophy Conditions

### Geostrophy refers to conditions where the Coriolis force is balanced by pressure forces

### This occurs when the Coriolis acceleration is greater than:  (i) advective acceleration, (ii) time rates of change, and (iii) viscous accelerations.  These conditions are generally codified in the following dimensionaless ratios:

### (i) $R_O \equiv \frac{U}{\Omega L} \ll 1$   [Rossby Number]

### (ii) $R_{OT} \equiv \frac{1}{\Omega T} \ll 1$  [Temporal Rossby Number]

### (iii) $E_K \equiv \frac{\nu_E}{\Omega H^2} \ll 1$   [Ekman Number]

#### where $U$ is the characterstic velocity, $\Omega$ is the angular veloctity, $T$ is the time scale, $L$ is the length scale, $H$ is the height scale and $\nu_E$ is the viscosity.

### Setup -- Rotating tank

In [11]:
U = 1 * unyt.cm/unyt.s  # flow velocity
Omega = 30. / unyt.minute   # rotation 
L = 20 * unyt.cm   # tank diameter
nu_E = 1e-6 * unyt.m**2/unyt.s # viscocity
H = 15 * unyt.cm # Height, although it may be only the upper surface matters

In [8]:
# Time
T = L/U
T

unyt_quantity(20., 's')

### Ratios

In [5]:
# Rossby
R_O = U / (Omega*L)
R_O

unyt_quantity(0.1, '(dimensionless)')

In [9]:
# Temporal Rossby 
R_OT = 1/(Omega * T)
R_OT

unyt_quantity(0.1, '(dimensionless)')

In [12]:
# Ekman
E_K = nu_E / (Omega * H**2)
E_K

unyt_quantity(8.88888889e-05, '(dimensionless)')

### All of these are less than (with $E_K \ll 1$) and geostrophy should hold

----

## PS2-2 Geostrophic current

### Setup

In [15]:
u = 5 * unyt.cm / unyt.s  # West to East
Delta_y = 200 * unyt.km
phi = -60 * unyt.deg # South

## Key Equation of Geostrophy


### $f u = - \frac{1}{\rho_0} \frac{\partial p}{\partial y}$

In [17]:
rho_0 = 1 * unyt.density_seawater
rho_0.to('kg/m**3')

unyt_quantity(1023.6, 'kg/m**3')

In [20]:
f = coriolis.coriolis_parameter(phi)
f.to('1/s')  # Note the negative sign

unyt_quantity(-0.0001263, '1/s')

### Rearranging

### $\frac{\partial p}{\partial y} = - \rho_0 f u$

In [22]:
# Calculate
dp_dy = -rho_0 * f * u
dp_dy.to('Pa/m')

unyt_quantity(0.0064642, 'Pa/m')

### Because $\partial p/\partial y > 0$, this implies higher $p$ to the North hence a higher height.

### We may estimate the height difference as

### $\Delta p = \rho_0 g \Delta h$

### or $\Delta h = \Delta p / (g \rho_0)$

### and recognize $\Delta p \approx (\partial p/\partial y) \Delta y$

In [23]:
# Calculate
g = 9.8 * unyt.m / unyt.s**2

Dh = dp_dy * Delta_y / (g * rho_0)
Dh.to('cm')

unyt_quantity(12.88809023, 'cm')

### We find the sea surface is $\approx 13$cm higher on the North side.

----

# PS2-3 (7-2)

## It is rather straightforward to identify the terms to re-insert the $f_*$ terms:

## $-f v + f_* w = -\frac{1}{\rho_0} \frac{\partial p}{\partial x}$

## $fu = -\frac{1}{\rho_0} \frac{\partial p}{\partial y}$

## $-f_* u = \frac{1}{\rho_0} \frac{\partial p}{\partial z}$

## We wish to show that motions around the axis of rotation (i.e. $u,v$, with each normal to the rotation vector $\bar \Omega$) are columnar.  That means $\partial u/\partial z = \partial v/\partial z = 0$

## I fear I can't quite intuit this one..

----

# PS2-4

## We begin by taking the curl of the horizontal momentum equations:

## $\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x}+ v \frac{\partial u}{\partial y} - fv = -g \frac{\partial \eta}{\partial x}$

## $\frac{\partial v}{\partial t} + u \frac{\partial v}{\partial x}+ v \frac{\partial v}{\partial y} + fu = -g \frac{\partial \eta}{\partial y}$

## This gives (before subtracting):

## $\frac{\partial^2 v}{\partial t \partial x} + \frac{\partial u}{\partial x} \frac{\partial v}{\partial x} + u \frac{\partial^2 v}{\partial x^2} + \frac{\partial v}{\partial x} \frac{\partial v}{\partial y} + v \frac{\partial^2 v}{\partial y \partial x} + f \frac{\partial u}{\partial x} + u \frac{\partial f}{\partial x} = -g \frac{\partial^2 \eta}{\partial y \partial x}$

## $\frac{\partial^2 u}{\partial t \partial y} + \frac{\partial u}{\partial x} \frac{\partial u}{\partial y} + u \frac{\partial^2 u}{\partial x \partial y} + \frac{\partial v}{\partial y} \frac{\partial u}{\partial y} + v \frac{\partial^2 u}{\partial y^2} - f \frac{\partial v}{\partial y} - v \frac{\partial f}{\partial y} = -g \frac{\partial^2 \eta}{\partial y \partial x}$

## Subtracting and grouping a few terms

## $\frac{\partial}{\partial t} (\frac{\partial v}{\partial x} - \frac{\partial u}{\partial y}) + f (\frac{\partial u}{\partial x} + \frac{\partial v}{\partial y})
+ (\frac{\partial u}{\partial x} + \frac{\partial v}{\partial y}) (\frac{\partial v}{\partial x} - \frac{\partial u}{\partial y}) + u (\frac{\partial^2 v}{\partial x^2} - \frac{\partial^2 v}{\partial x \partial y}) + v (\frac{\partial^2 v}{\partial x \partial y} - \frac{\partial^2 u}{\partial y^2}) + u \frac{\partial f}{\partial x} + v \frac{\partial f}{\partial y} = 0$

## Which, guided by C-R section 7.4 may be re-written as:

## $\frac{d}{dt} (f + \frac{\partial v}{\partial x} - \frac{\partial u}{\partial y})
+ (\frac{\partial u}{\partial x} + \frac{\partial v}{\partial y}) (f + \frac{\partial v}{\partial x} - \frac{\partial u}{\partial y}) = 0$

## or 

## $\frac{d}{dt} (f + \zeta)
+ (\frac{\partial u}{\partial x} + \frac{\partial v}{\partial y}) (f + \zeta) = 0$

## Now, we examine the continuity equation:

## $\frac{\partial h}{\partial t} + \frac{\partial}{\partial x} (hu) + \frac{\partial}{\partial y} (hv) = 0$

## Insert the material time derivative:

## $\frac{d h}{dt} = \frac{\partial h}{\partial t} + u \frac{\partial h}{\partial x} + v \frac{\partial h}{\partial y}$

## And rearrange to get:

## $\frac{d h}{dt} + (\frac{\partial u}{\partial x} + \frac{\partial v}{\partial y}) h = 0$

## I now follow C-R (shamelessly) to introduce a incompressible fluid column of cross-section $ds$ with volume $hds$.  Its incompressibility implies:

## $\frac{d}{dt} (h \, ds) = 0$

## Expanding, 

## $\frac{dh}{dt} ds + h \frac{d}{dt} ds = 0$

## Using our re-write of the continuity equation from above, we recover:

## $\frac{d}{dt} ds = (\frac{\partial u}{\partial x} + \frac{\partial v}{\partial y}) ds$

## Combining the above with the equation derived from the curl, gives:

## $\frac{d}{dt} [(f+\zeta)ds] = 0$

## And as we have already shown $d/dt (hds) = 0$, the ratio of each will also be 0

## $\frac{d}{dt} \left (\frac{f + \zeta}{h} \right ) = 0$

## or

## $\frac{Dq}{Dt} = 0$

----

# PS2-5 (CR 7-6) -- Jet flow

### In the following, we will adopt our continuity equation for the shallow-water model and also conserve potential vorticity, i.e.

### $\frac{\partial \eta}{\partial t} + \frac{\partial}{\partial x} (hu) + \frac{\partial}{\partial y} (hv) = 0$

### and

### $q = \frac{f + \zeta}{h}$ is a constant

### We will set a coordinate system where the jet is travelling along $\hat x$ and where $\Delta U / \Delta y \approx U_1/L_1 > 0$, as per the figure in C-R.

In [25]:
# Setup
H1 = 200 * unyt.m
H2 = 160 * unyt.m
U1 = 0.5 * unyt.m / unyt.s
L1 = 10 * unyt.km
f = 1e-4 / unyt.s

### Is this flow geostropic?

In [26]:
R_O = (U1/L1) / f
R_O
# E_k is satisfied as we are frictionless

unyt_quantity(0.5, '(dimensionless)')

### So we are marginally geostrophic

### Can we apply the rigid lid approximation?

In [27]:
f**2 * L1**2 / (9.8*unyt.m/unyt.s**2 * H1)   # Equation 7.31 of C-R

unyt_quantity(0.0005102, '(dimensionless)')

### We can!  Therefore $\partial \eta / \partial t$ may be neglected and we have:

### $\frac{\partial}{\partial x} (hu) + \frac{\partial}{\partial y} (hv) = 0$

### As we have a jet with $v \approx 0$, we will neglect the 2nd term, leaving:

### $\frac{\partial}{\partial x} (hu) = 0$

### So, as we cross the escarpment, we expect:

###  $H_1 U_1 \approx H_2 U_2$

In [28]:
U2 = U1 * H1 / H2
U2

unyt_quantity(0.625, 'm/s')

### Last, we conserve potential vorticity

### $q = \frac{f + \zeta}{h} = \frac{f + \zeta_1}{H_1} = \frac{f + \zeta_2}{H_2}$

### with $\zeta = \partial v / \partial x - \partial u / \partial y$

### Again, we will ignore terms related to $v$ giving:

### $\zeta \approx -\partial u / \partial y$

### Therefore,   $\frac{f - U_1/L_1}{H_1} = \frac{f - U_2/L_2}{H_2}$

### Rearranging, we have:

### $L_2 = \frac{U_2}{(f/H_2 - f/H_1 + U_1/L_1 H_1) H_2}$


In [29]:
L2 = U2 / (f/H2 - f/H1 + U1/L1/H1) / H2
L2.to('km')

unyt_quantity(10.41666667, 'km')

### If $H_2 = 100$m, $U_2$ and $L_2$ would be yet larger.

----

# PS2-6 Extra Credit

## Will work on this tomorrow